In [ ]:
%load_ext autoreload
%autoreload 2

import evolution
from config import Config
from pymoo.algorithms.moo.nsga2 import NSGA2
import os
import evolution, evolution_operations
from pymoo.optimize import minimize

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

config_settings = {
    'n_gen': 1,
    'dataset': 'mnist',
    'pop_size': 2,
    'n_epochs': 1,
    'number_of_objectives': 1}

iterations = 1
iteration_for_second_objective = 2

for i in range(iterations):
    if i == iteration_for_second_objective: config_settings['obj_num'] = 2

    config = Config(config_settings)      
    problem = evolution.EVProblem(config)
    algorithm = NSGA2(pop_size=config.pop_size,
                sampling=evolution_operations.SamplingFromSmall(),
                mutation=evolution_operations.MutationFromSmall(),
                eliminate_duplicates=True)
    res = minimize(problem,
                    algorithm, 
                    callback=evolution.do_every_generations,
                    termination=config.termination)


In [ ]:
algorithm

In [ ]:
res.pop.get("X")